# Inference
As for training, we use `gunpowder` for predicting a larger area.
## Exercise 1 : Start Inference
1. Stop first training in `01_training.ipynb`
2. Your task is to run the script twice, one time with `datasetsize` set to small, and one to big. With this we get two affinity predictions that we can play with in `03_agglomeration.ipynb`
2. The default model that we use to make our prediction is a pretrained network. Change the `setup_dir`, such that it points to your training directory. By doing that, the network's weights are loaded from your model instead from our pretrained model. You also need to adapt `iteration` depending on what the last tensorflow checkpoint was, that are written out. For this go to your training directory, and search for this file: ``

Tip: It takes a while for the script to start a prediction. After aorund half a minute, you should be able to see following output. If you wait 2mins and nothing happens, restart the kernel and try again.
```
INFO:gunpowder.nodes.scan:processed chunk 1/288
INFO:gunpowder.nodes.scan:processed chunk 2/288
INFO:gunpowder.nodes.scan:processed chunk 3/288
INFO:gunpowder.nodes.scan:processed chunk 4/288
```


In [1]:
from __future__ import print_function
from gunpowder import *
from gunpowder.tensorflow import *
import json
import numpy as np
import os
import sys
import logging
import daisy
import os
import time


def predict(iteration, in_file, out_file, setup_dir,
            out_dataset, datasetsize='small'):

    with open(os.path.join(setup_dir, 'train_net.json'), 'r') as f:
        config = json.load(f)

    # voxels
    voxel_size = Coordinate((40, 4, 4))
    input_shape = Coordinate(config['input_shape']) # Network specific
    output_shape = Coordinate(config['output_shape'])
    context = (input_shape - output_shape) // 2
    
    # Initial ROI
    # got nice offset visually from neuroglancer : 1507, 1678, 100
    if datasetsize == 'big':
        offset = (50, 1400, 1400) * np.array(voxel_size)  # avoid big glia and cell body in sample 0
        chunkgrid = [2, 12, 12]

    elif datasetsize == 'small':
        offset = (90, 1600, 1400) * np.array(voxel_size)  
        chunkgrid = [1, 5, 5]


    roi = Roi(
        offset=offset,
        shape=output_shape * voxel_size * Coordinate(chunkgrid),
    )


    # nm
    context_nm = context * voxel_size
    read_roi = roi.copy()
    read_roi = read_roi.grow(context_nm, context_nm)

    input_size = input_shape * voxel_size
    output_size = output_shape * voxel_size

    output_roi = read_roi.grow(-context_nm, -context_nm)
    print("Read ROI in nm is %s" % read_roi)
    print("Output ROI in nm is %s" % output_roi)

    print("Read ROI in voxel space is {}".format(read_roi / voxel_size))
    print("Output ROI in voxel space is {}".format(output_roi / voxel_size))

    raw = ArrayKey('RAW')
    affs = ArrayKey('AFFS')

    output_roi = daisy.Roi(
        output_roi.get_begin(),
        output_roi.get_shape()
    )


    # Caution: daisy.ROI and gunpowder.Roi have different behaviour, Prepare_ds only works with daisy.Roi, while
    # gunpowder node eg. Crop Node only works with gunpowder.Roi
    ds = daisy.prepare_ds(
        out_file,
        out_dataset,
        output_roi,
        voxel_size,
        'float32',
        # write_size=output_size,
        write_roi=daisy.Roi((0, 0, 0), output_size),
        num_channels=3,
        # temporary fix until
        # https://github.com/zarr-developers/numcodecs/pull/87 gets approved
        # (we want gzip to be the default)
        compressor={'id': 'gzip', 'level': 5}
    )

    chunk_request = BatchRequest()
    chunk_request.add(raw, input_size)
    chunk_request.add(affs, output_size)

    pipeline = (
            N5Source(
                in_file,
                datasets={
                    raw: 'volumes/raw'
                },
            ) +
            Pad(raw, size=None) +
            Crop(raw, read_roi) +
            Normalize(raw) +
            IntensityScaleShift(raw, 2, -1) +
            Predict(
                os.path.join(setup_dir, 'train_net_checkpoint_%d' % iteration),
                inputs={
                    config['raw']: raw
                },
                outputs={
                    config['affs']: affs
                },
                # TODO: change to predict graph
                graph=os.path.join(setup_dir, 'train_net.meta')
            ) +
            IntensityScaleShift(raw, 0.5, 0.5) +  # Just for visualization.
            ZarrWrite(
                dataset_names={
                    affs: out_dataset,
                    raw: 'volumes/raw',
                },
                output_filename=out_file
            ) +  # TODO: Would be nice to have a consistent file format (eg. only n5)
            PrintProfilingStats(every=10) +
            Scan(chunk_request)
    )
    start_time = time.time()
    print("Starting prediction...")
    with build(pipeline):
        pipeline.request_batch(BatchRequest())
    print("Prediction finished in {:0.2f}".format(time.time() - start_time))


In [2]:
logging.basicConfig(level=logging.INFO)
logging.getLogger('gunpowder.nodes.hdf5like_write_base').setLevel(
    logging.INFO)

in_file = '../../jan/segmentation/data/sample_0.n5'  # This is our raw file
setup_dir = '../../jan/segmentation/snapshots/setup58_p/' # This is the pretrained model. 


out_dataset = 'volumes/affs'
datasetsize = 'big' # choose big or small
iteration = 500000

out_file = 'affinities_{}_{:05}.zarr'.format(datasetsize, iteration)

predict(
    iteration,
    in_file,
    out_file,
    setup_dir,
    out_dataset,
    datasetsize=datasetsize)

INFO:daisy.datasets:Reusing existing dataset


Read ROI in nm is [1280:6560, 5176:8712, 5176:8712] (5280, 3536, 3536)
Output ROI in nm is [2000:5840, 5600:8288, 5600:8288] (3840, 2688, 2688)
Read ROI in voxel space is [32:164, 1294:2178, 1294:2178] (132, 884, 884)
Output ROI in voxel space is [50:146, 1400:2072, 1400:2072] (96, 672, 672)


INFO:gunpowder.tensorflow.local_server:Creating local tensorflow server
INFO:gunpowder.tensorflow.local_server:Server running at b'grpc://localhost:43233'
INFO:gunpowder.tensorflow.nodes.predict:Initializing tf session, connecting to b'grpc://localhost:43233'...
INFO:gunpowder.tensorflow.nodes.predict:Reading graph from ../../jan/segmentation/snapshots/setup58_p/train_net.meta and weights from ../../jan/segmentation/snapshots/setup58_p/train_net_checkpoint_500000...


INFO:tensorflow:Restoring parameters from ../../jan/segmentation/snapshots/setup58_p/train_net_checkpoint_500000


INFO:tensorflow:Restoring parameters from ../../jan/segmentation/snapshots/setup58_p/train_net_checkpoint_500000
INFO:gunpowder.nodes.scan:scanning over 288 chunks


...

INFO:gunpowder.nodes.scan:processed chunk 100/288
INFO:gunpowder.nodes.scan:processed chunk 101/288
INFO:gunpowder.nodes.scan:processed chunk 102/288
INFO:gunpowder.nodes.scan:processed chunk 103/288
INFO:gunpowder.nodes.scan:processed chunk 104/288
INFO:gunpowder.nodes.scan:processed chunk 105/288
INFO:gunpowder.nodes.scan:processed chunk 106/288
INFO:gunpowder.nodes.scan:processed chunk 107/288
INFO:gunpowder.nodes.scan:processed chunk 108/288
INFO:gunpowder.nodes.scan:processed chunk 109/288
INFO:gunpowder.nodes.print_profiling_stats:
Profiling Stats

NODE                METHOD    COUNTS    MIN       MAX       MEAN      MEDIAN    
Crop                prepare   10        0.00      0.00      0.00      0.00      
Crop                process   10        0.00      0.00      0.00      0.00      
IntensityScaleShift prepare   20        0.00      0.00      0.00      0.00      
IntensityScaleShift process   20        0.01      0.02      0.01      0.01      
N5Source                      10  

INFO:gunpowder.nodes.scan:processed chunk 150/288
INFO:gunpowder.nodes.scan:processed chunk 151/288
INFO:gunpowder.nodes.scan:processed chunk 152/288
INFO:gunpowder.nodes.scan:processed chunk 153/288
INFO:gunpowder.nodes.scan:processed chunk 154/288
INFO:gunpowder.nodes.scan:processed chunk 155/288
INFO:gunpowder.nodes.scan:processed chunk 156/288
INFO:gunpowder.nodes.scan:processed chunk 157/288
INFO:gunpowder.nodes.scan:processed chunk 158/288
INFO:gunpowder.nodes.scan:processed chunk 159/288
INFO:gunpowder.nodes.print_profiling_stats:
Profiling Stats

NODE                METHOD    COUNTS    MIN       MAX       MEAN      MEDIAN    
Crop                prepare   10        0.00      0.00      0.00      0.00      
Crop                process   10        0.00      0.00      0.00      0.00      
IntensityScaleShift prepare   20        0.00      0.00      0.00      0.00      
IntensityScaleShift process   20        0.01      0.02      0.01      0.01      
N5Source                      10  

INFO:gunpowder.nodes.scan:processed chunk 200/288
INFO:gunpowder.nodes.scan:processed chunk 201/288
INFO:gunpowder.nodes.scan:processed chunk 202/288
INFO:gunpowder.nodes.scan:processed chunk 203/288
INFO:gunpowder.nodes.scan:processed chunk 204/288
INFO:gunpowder.nodes.scan:processed chunk 205/288
INFO:gunpowder.nodes.scan:processed chunk 206/288
INFO:gunpowder.nodes.scan:processed chunk 207/288
INFO:gunpowder.nodes.scan:processed chunk 208/288
INFO:gunpowder.nodes.scan:processed chunk 209/288
INFO:gunpowder.nodes.print_profiling_stats:
Profiling Stats

NODE                METHOD    COUNTS    MIN       MAX       MEAN      MEDIAN    
Crop                prepare   10        0.00      0.00      0.00      0.00      
Crop                process   10        0.00      0.00      0.00      0.00      
IntensityScaleShift prepare   20        0.00      0.00      0.00      0.00      
IntensityScaleShift process   20        0.01      0.02      0.01      0.01      
N5Source                      10  

INFO:gunpowder.nodes.scan:processed chunk 250/288
INFO:gunpowder.nodes.scan:processed chunk 251/288
INFO:gunpowder.nodes.scan:processed chunk 252/288
INFO:gunpowder.nodes.scan:processed chunk 253/288
INFO:gunpowder.nodes.scan:processed chunk 254/288
INFO:gunpowder.nodes.scan:processed chunk 255/288
INFO:gunpowder.nodes.scan:processed chunk 256/288
INFO:gunpowder.nodes.scan:processed chunk 257/288
INFO:gunpowder.nodes.scan:processed chunk 258/288
INFO:gunpowder.nodes.scan:processed chunk 259/288
INFO:gunpowder.nodes.print_profiling_stats:
Profiling Stats

NODE                METHOD    COUNTS    MIN       MAX       MEAN      MEDIAN    
Crop                prepare   10        0.00      0.00      0.00      0.00      
Crop                process   10        0.00      0.00      0.00      0.00      
IntensityScaleShift prepare   20        0.00      0.00      0.00      0.00      
IntensityScaleShift process   20        0.01      0.02      0.01      0.01      
N5Source                      10  

Prediction finished in 322.78


## Exercise 2 : Throughput calculations
1. While you are waiting for the prediction to finish, you can calculate how many chunks needed to be predicted, if we were to process an entire fly brain
  - the total size of the predicted dataset=big --> (3840, 2688, 2688) nm
  - this translates to 288 chunks
  - the dimension of the FAFB, a full adult fly brain dataset are: (282, 519, 991) microns

2. After the prediction, you can make an estimate for how long it would take, if we only had a single gpu available

In [4]:
# Type your solution here
# -------

## Exercise 3 : Check predicted affinities in neuroglancer
1. Check out whether your prediction was successfull by exploring the result in `neuroglancer`
2. If you are satisfied, you can move on to `03_agglomeration.ipynb`. If you have the feeling, your predictions are not correct (eg. only black), or you cant see any neuron border, consider to use the pretrained network instead of your own model. You need ok-affinity predictions for the next notebook.
3. You might want to come back to this notebook, if you are interested in comparing eg. your model and our pretrained model.

In [11]:
import neuroglancer
import funlib.show.neuroglancer as funlibng

ip_address = ''
neuroglancer.set_server_bind_address('0.0.0.0')
viewer = neuroglancer.Viewer()

raw_ds = daisy.open_ds(in_file, 'volumes/raw')
affs_ds = daisy.open_ds(out_file, out_dataset)

with viewer.txn() as s:
    funlibng.add_layer(s, raw_ds, 'raw')
    funlibng.add_layer(s, affs_ds, 'affs', shader='rgb')
if len(ip_address) == 0:
    print('you first have to set the ip address of your paperspace machine ')
else:
    print(viewer.__str__().replace('localhost', ip_address))

you first have to set the ip address of your paperspace machine 


### Help & Troubleshooting
- If your notebook is hanging, sometimes it helps to restart the kernel: For this go to `Kernel` and then `Restart & Clear Output`
- If you get an error message: `compatible`, remove the container/file, that you are trying to write to. For instance, remove `affinities_big_50000.zarr` from the current directory if you have set this as `out_file`.
- We observed, that after prediction is done, the notebook might die, so the output disappears, and it might seem to you, that nothing worked out, but this is not true. It usually dies, after the prediction has finished.